In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Importando dataset  do meu google drive
from google.colab import drive 
drive.mount('/content/gdrive')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive


In [ ]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Inteligência computacional/Competição 1/train.csv')

In [ ]:
def preproc(df):
    df.QT_DIA_SOLICITADO = df.QT_DIA_SOLICITADO.fillna(0)
    df.DS_INDICACAO_ACIDENTE = df.DS_INDICACAO_ACIDENTE.fillna('0')
    df.DS_TIPO_INTERNACAO = df.DS_TIPO_INTERNACAO.fillna('0')
    df.DS_TIPO_ATENDIMENTO = df.DS_TIPO_ATENDIMENTO.fillna('0')
    df.DS_UNIDADE_TEMPO_DOENCA = df.DS_UNIDADE_TEMPO_DOENCA.fillna('0')
    df.QT_TEMPO_DOENCA = df.QT_TEMPO_DOENCA.fillna(0)
    df.CD_GUIA_REFERENCIA = df.CD_GUIA_REFERENCIA.fillna(0)
    return df

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np

def standardScalerFunc(data):
  ss = StandardScaler()
  sstransformed = ss.fit_transform(data)

  return ss, pd.DataFrame(sstransformed)

def labelEncoderFunc(data):
  ss = LabelEncoder()
  sstransformed = ss.fit_transform(data) 

  return ss, pd.DataFrame(sstransformed)

def oneHotEncoderFunc(data):
  ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
  ohetransformed = ohe.fit_transform(data)
  
  return ohe, pd.DataFrame(ohetransformed)

In [ ]:
df_train = preproc(df_train)
numeric = df_train[['NR_SEQ_REQUISICAO', 'QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']]
categoric = df_train[['DS_ITEM','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO', 'CD_ITEM', 'DS_CBO', 'DS_SUBGRUPO']]
target = df_train['DS_STATUS_ITEM']

X = pd.concat([numeric, categoric], axis=1)
print(len(X))
X = X.dropna()
print(len(X))
y = target

227122
227122


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # 80% training 20% test

#train
ss, x_train_ss = standardScalerFunc(X_train[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
ohe, x_train_ohe = oneHotEncoderFunc(X_train[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])

x_train = pd.concat([x_train_ss, x_train_ohe], axis=1)

#test
x_test_ss = ss.transform(X_test[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
x_test_ohe = ohe.transform(X_test[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])

x_test = pd.concat([pd.DataFrame(x_test_ss), pd.DataFrame(x_test_ohe)], axis=1)

#classifier
classifier = svm.SVC()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
ss, x_ss = standardScalerFunc(X[['QT_SOLICITADA', 'QT_DIA_SOLICITADO', 'QT_TEMPO_DOENCA', 'CD_GUIA_REFERENCIA']])
ohe, x_ohe = oneHotEncoderFunc(X[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE', 'DS_TIPO_GUIA', 'DS_GRUPO']])
x = pd.concat([pd.DataFrame(x_ss), pd.DataFrame(x_ohe)], axis=1)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C':[0.1, 1, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

grid = GridSearchCV(svm.SVC()), param_grid, verbose=3, return_train_score=True, cv=3)
grid.fit(x, y)
print(f'Parâmetros encontrados: {grid.best_params_}')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END max_depth=25, max_features=10;, score=(train=0.754, test=0.720) total time=  21.3s
[CV 2/3] END max_depth=25, max_features=10;, score=(train=0.756, test=0.718) total time=  21.1s
[CV 3/3] END max_depth=25, max_features=10;, score=(train=0.755, test=0.716) total time=  21.1s
[CV 1/3] END max_depth=25, max_features=7;, score=(train=0.753, test=0.720) total time=  21.2s
[CV 2/3] END max_depth=25, max_features=7;, score=(train=0.755, test=0.718) total time=  20.0s
[CV 3/3] END max_depth=25, max_features=7;, score=(train=0.755, test=0.716) total time=  20.3s
[CV 1/3] END max_depth=25, max_features=5;, score=(train=0.752, test=0.719) total time=  19.3s
[CV 2/3] END max_depth=25, max_features=5;, score=(train=0.754, test=0.717) total time=  19.9s
[CV 3/3] END max_depth=25, max_features=5;, score=(train=0.754, test=0.716) total time=  19.8s
[CV 1/3] END max_depth=25, max_features=3;, score=(train=0.750, test=0.719) total